<a href="https://colab.research.google.com/github/tadinve/rsa_on_aws/blob/main/notebook/01_Cadabra_Data_Ingestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0 - AWS Setup


1. Do this only Once
- Create an account in AWS
- Create an IAM User and downlod credentials file (you will need access and secret key)
- run aws init
- copy credentials file to AWS_CRED_DIR


In [ ]:
%%bash
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -qq awscliv2.zip
sudo ./aws/install

You can now run: /usr/local/bin/aws --version


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.4M  100 41.4M    0     0   172M      0 --:--:-- --:--:-- --:--:--  172M


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Define Directory Variables

In [ ]:
def escapeSlash(s):
    return s.replace("/","\/")

In [ ]:
#python variables
HOME = "/root"
AWS_CRED_DIR = "/gdrive/MyDrive/AWS/ca_dev/"
AWS_CRED_FILE = AWS_CRED_DIR + "credentials"
AWS_CONFIG_FILE = AWS_CRED_DIR + "config"
TF_DIR = "/gdrive/MyDrive/AWS/AWS-TF/Cadabra"
TRANSACTIONS_DIR="/data/"
NAME_INITIALS = "" #ADD YOUR NAME INTIALS **********************************

In [ ]:
#shell variables using python variables
%env AWS_CRED_DIR={AWS_CRED_DIR}
%env AWS_CRED_FILE={AWS_CRED_FILE}

env: AWS_CRED_DIR=/gdrive/MyDrive/AWS/ca_dev/
env: AWS_CRED_FILE=/gdrive/MyDrive/AWS/ca_dev/credentials


## Check for Credentials file

https://linuxize.com/post/bash-check-if-file-exists/

In [ ]:
%%bash
if [ ! -f "$AWS_CRED_FILE" ]; then 
    aws configure 
fi

In [ ]:
!mkdir -p ~/.aws
!cp {AWS_CRED_DIR}c* ~/.aws

In [ ]:
#Read the credentials
f = open(AWS_CRED_FILE)
header = f.readline()
line = f.readline().split("=")
access_key=line[1].strip()
line = f.readline().split("=")
secret_key=escapeSlash(line[1].strip())
f.close()
f = open(AWS_CONFIG_FILE)
header = f.readline()
line = f.readline().split("=")
user_region=line[1].strip()

In [ ]:
#import os
#os.environ["AWS_ACCESS_KEY_ID"]=access_key
#os.environ["AWS_SECRET_ACCESS_KEY"]=secret_key
#os.environ["AWS_DEFAULT_REGION"]=user_region
#os.environ['AWS_PROFILE'] = "default"

# Install Kinesis & Start Agent



## Install Java 1.8
- needed by Kinesis build
- from https://stackoverflow.com/questions/58106622/how-to-change-the-java-version-in-google-colab

In [ ]:
%cd {HOME}

/root


In [ ]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://security.ubuntu.com/ubu

## Get Kinesis Code from Github and Build

In [ ]:
cd {HOME}

/root


In [ ]:
KINESIS_AGENT_VER = "1.1.4"

In [ ]:
!curl -LO https://github.com/awslabs/amazon-kinesis-agent/archive/{KINESIS_AGENT_VER}.tar.gz
!tar -xvzf {KINESIS_AGENT_VER}.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   135  100   135    0     0   1500      0 --:--:-- --:--:-- --:--:--  1500
100  238k    0  238k    0     0   910k      0 --:--:-- --:--:-- --:--:-- 1849k
amazon-kinesis-agent-1.1.4/
amazon-kinesis-agent-1.1.4/.gitignore
amazon-kinesis-agent-1.1.4/.travis.yml
amazon-kinesis-agent-1.1.4/LICENSE.txt
amazon-kinesis-agent-1.1.4/NOTICE.txt
amazon-kinesis-agent-1.1.4/README.md
amazon-kinesis-agent-1.1.4/bin/
amazon-kinesis-agent-1.1.4/bin/aws-kinesis-agent-babysit
amazon-kinesis-agent-1.1.4/bin/aws-kinesis-agent-manual-run.sh
amazon-kinesis-agent-1.1.4/bin/aws-kinesis-agent.Debian
amazon-kinesis-agent-1.1.4/bin/aws-kinesis-agent.RedHat
amazon-kinesis-agent-1.1.4/bin/aws-kinesis-agent.Ubuntu
amazon-kinesis-agent-1.1.4/bin/start-aws-kinesis-agent
amazon-kinesis-agent-1.1.4/build.xml
amazon-kinesis-agent-1.1.4/configuration/
amazon-kine

In [ ]:
!mv amazon-kinesis-agent-{KINESIS_AGENT_VER} amazon-kinesis-agent
%cd amazon-kinesis-agent
!sudo ./setup --install

/root/amazon-kinesis-agent
Detected OS Distro: Debian
Uninstalling aws-kinesis-agent ...
Removing aws-kinesis-agent from system services...
userdel: user 'aws-kinesis-agent-user' does not exist
Installing Kinesis Agent ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  ant-optional
Suggested packages:
  ant-doc antlr javacc junit junit4 jython libactivation-java libbcel-java
  libbsf-java libcommons-logging-java libcommons-net-java libmail-java
  libjaxp1.3-java libjdepend-java libjsch-java liblog4j1.2-java liboro-java
  libregexp-java libxalan2-java libxml-commons-resolver1.1-java libxz-java
The following NEW packages will be installed:
  ant ant-optional
0 upgraded, 2 newly installed, 0 to remove and 85 not upgraded.
Need to get 2,453 kB of arc

## Configure Kinesis Agent

In [ ]:
cd /etc/aws-kinesis

/etc/aws-kinesis


In [ ]:
%%writefile agent.json
{
  "cloudwatch.emitMetrics": true,
  "awsAccessKeyId": "ACCESS_KEY",
  "awsSecretAccessKey": "SECRET_KEY",
  "firehose.endpoint": "firehose.USER_REGION.amazonaws.com",
  "kinesis.endpoint": "kinesis.USER_REGION.amazonaws.com",
 
  "flows": [
    {
      "filePattern": "/data/*.log",
      "deliveryStream": "tf-cadabra_batch_sales",
      "initialPosition": "START_OF_FILE"
    },
    {
      "filePattern": "/data/*.log",
      "kinesisStream": "tf-order-stream",
      "partitionKeyOption": "RANDOM",
      "dataProcessingOptions": [
         {
            "optionName": "CSVTOJSON",
            "customFieldNames": ["InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "Customer", "Country"]
         }
      ]
    },
    {
      "filePattern": "/var/log/httpd/*.log",
      "deliveryStream": "tf-weblogs",
      "initialPosition": "START_OF_FILE"
    },
    {
      "filePattern": "/data/*.log",
      "kinesisStream": "tf-redshift-datastream",
      "partitionKeyOption": "RANDOM"
    }
  ]
}

Overwriting agent.json


In [ ]:
!sed -i 's/USER_REGION/{user_region}/g' agent.json
!sed -i 's/ACCESS_KEY/{access_key}/g' agent.json
!sed -i 's/SECRET_KEY/{secret_key}/g' agent.json

In [ ]:
!cat agent.json

In [ ]:
!ls -alh /etc/aws-kinesis/ 

total 20K
drwxr-xr-x 3 root root 4.0K May  4 19:06 .
drwxr-xr-x 1 root root 4.0K May  4 19:06 ..
drwxr-xr-x 2 root root 4.0K May  4 19:06 agent.d
-rw-r--r-- 1 root root 1.1K May  4 19:06 agent.json


## Start Kinesis Agent

In [ ]:
!sudo service aws-kinesis-agent status

 * aws-kinesis-agent is not running


In [ ]:
!sudo service aws-kinesis-agent start

 * Starting aws-kinesis-agent
   ...done.


In [ ]:
!sleep 30

In [ ]:
!cat /var/log/aws-kinesis-agent/aws-kinesis-agent.log  

2021-05-04 19:07:02.212+0000 704287a1c18b (main) com.amazon.kinesis.streaming.agent.Agent [INFO] Reading configuration from file: /etc/aws-kinesis/agent.json
2021-05-04 19:07:02.213+0000 704287a1c18b (main) com.amazon.kinesis.streaming.agent.Agent [INFO] Found 4 configured flow(s)
2021-05-04 19:07:02.249+0000 704287a1c18b (main) com.amazon.kinesis.streaming.agent.Agent [INFO] null: Agent will use up to 24 threads for sending data.
2021-05-04 19:07:02.378+0000 704287a1c18b (main) com.amazon.kinesis.streaming.agent.tailing.checkpoints.SQLiteFileCheckpointStore [INFO] Deleted 0 old checkpoints.
2021-05-04 19:07:02.388+0000 704287a1c18b (Agent STARTING) com.amazon.kinesis.streaming.agent.Agent [INFO] Agent: Starting up...
2021-05-04 19:07:02.392+0000 704287a1c18b (Agent STARTING) com.amazon.kinesis.streaming.agent.Agent [INFO] Agent: Starting tailer for file fh:tf-cadabra_batch_sales:/data/*.log
2021-05-04 19:07:02.410+0000 704287a1c18b (Agent STARTING) com.amazon.kinesis.streaming.agent.A

# Generate Data

In [ ]:
!mkdir -p {TRANSACTIONS_DIR}
%cd {HOME}

/root


## Get Requirements

In [ ]:
#get requirements.txt
!wget --no-check-certificate -q 'https://docs.google.com/uc?export=download&id=1-HiavLHD1DPYov9YVyH-xJ4ggdN4AahF' -O requirements.txt

In [ ]:
!pip install -r requirements.txt 

     |████████████████████████████████| 133kB 15.8MB/s 
     |████████████████████████████████| 1.2MB 23.4MB/s 
     |████████████████████████████████| 7.5MB 34.6MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 153kB 44.9MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## Download Source Files and Data Template

In [ ]:
!ls -ltr

total 248
-rw-r--r-- 1 root root 244258 Apr 19 00:10 1.1.4.tar.gz
drwxrwxr-x 9 root root   4096 Apr 19 00:10 amazon-kinesis-agent
-rw-r--r-- 1 root root     11 Apr 19 00:10 requirements.txt


In [ ]:
!wget --no-check-certificate -q 'https://docs.google.com/uc?export=download&id=1gDonM3p8OKk9nKCb853N3jxLZ5qsE9pF' -O cadabra_products.csv

In [ ]:
!wget https://raw.githubusercontent.com/tadinve/rsa_on_aws/main/src/python_code/transaction_generator.py -O transaction_generator.py

--2021-05-04 19:12:50--  https://raw.githubusercontent.com/tadinve/rsa_on_aws/main/src/python_code/transaction_generator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2313 (2.3K) [text/plain]
Saving to: ‘transaction_generator.py’

transaction_generat 100%[===================>]   2.26K  --.-KB/s    in 0s      

2021-05-04 19:12:50 (36.5 MB/s) - ‘transaction_generator.py’ saved [2313/2313]



In [ ]:
!wget --no-check-certificate -q 'https://raw.githubusercontent.com/tadinve/rsa_on_aws/main/src/python_code/create_redshift_tables.py' -O create_redshift_tables.py

In [ ]:
#Create Target Schema and Tables in Redshift
from create_redshift_tables import create_schema_tables
create_schema_tables('dev','awsuser')

{'ClusterIdentifier': 'tf-redshift-cluster', 'CreatedAt': datetime.datetime(2021, 5, 4, 19, 12, 58, 891000, tzinfo=tzlocal()), 'Database': 'dev', 'DbUser': 'awsuser', 'Id': 'a98540de-6892-47ac-ae0b-1da0aaac5439', 'ResponseMetadata': {'RequestId': 'ba9118e6-5c82-497c-8dc6-8b119b5e0129', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ba9118e6-5c82-497c-8dc6-8b119b5e0129', 'content-type': 'application/x-amz-json-1.1', 'content-length': '152', 'date': 'Tue, 04 May 2021 19:12:59 GMT'}, 'RetryAttempts': 0}}
{'ClusterIdentifier': 'tf-redshift-cluster', 'CreatedAt': datetime.datetime(2021, 5, 4, 19, 13, 4, 166000, tzinfo=tzlocal()), 'Database': 'dev', 'DbUser': 'awsuser', 'Id': 'f603d210-0527-4c42-8b8f-c107464d0370', 'ResponseMetadata': {'RequestId': 'e7d567e6-fdb9-4eae-bff8-24abfaa1e78a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e7d567e6-fdb9-4eae-bff8-24abfaa1e78a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '152', 'date': 'Tue, 04 May 202

In [ ]:
!wget -O access.log.zip https://www.dropbox.com/s/8spgz9gfbeaa34k/KagglWebLogs.zip

--2021-04-19 00:10:59--  https://www.dropbox.com/s/8spgz9gfbeaa34k/KagglWebLogs.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8spgz9gfbeaa34k/KagglWebLogs.zip [following]
--2021-04-19 00:10:59--  https://www.dropbox.com/s/raw/8spgz9gfbeaa34k/KagglWebLogs.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0f65ceb1163ee37e6d8637520d.dl.dropboxusercontent.com/cd/0/inline/BM6wQwrkoflyL5jQy-HnIYpsEq2bTTBKHOXoOBX1wJ2aippq23k3CdVgNOVrQ4LCkxtfEkCniLy6SehBop84TP7WadcC5dtkwcZkU0XbJNgW20UFCRRuSpK704BGacBiCLfX_-dMpeLifadG_76oLHEp/file# [following]
--2021-04-19 00:10:59--  https://uc0f65ceb1163ee37e6d8637520d.dl.dropboxusercontent.com/cd/0/inline/BM6wQwrkoflyL5jQy-HnIYpsEq2bTTBKHOXoOBX1wJ2aippq23k3CdVgNOVrQ4LCkxtfEkCniLy6Se

## Generate Transactions

In [ ]:
from transaction_generator import generateTransactions
outfile = TRANSACTIONS_DIR+"test.log"
generateTransactions(n=2500,OutFile=outfile)

In [ ]:
!head {outfile}

0,22817,CARD SUKI BIRTHDAY,1,01/01/2018 00:00:00,0.45,2810849,Palau
1,21794,CLASSIC FRENCH STYLE BASKET NATURAL,7,01/01/2018 11:20:15,2.62,3626622,Pakistan
2,35915C,PEACH KNITTED HEN ,0,01/01/2018 22:40:30,1.36,5424886,Greenland
3,85150,LADIES & GENTLEMEN METAL SIGN,0,01/02/2018 10:00:46,4.22,5205044,Ireland
4,23035,DRAWER KNOB CERAMIC IVORY,17,01/02/2018 21:21:01,2.09,8149855,Sudan
5,21508,VINTAGE KID DOLLY CARD ,11,01/03/2018 08:41:16,0.44,6165043,French Polynesia
6,21731,RED TOADSTOOL LED NIGHT LIGHT,-7,01/03/2018 20:01:32,2.62,7330086,Saint Kitts and Nevis
7,22028,PENNY FARTHING BIRTHDAY CARD,-11,01/04/2018 07:21:47,0.70,6268386,Montserrat
8,22192,BLUE DINER WALL CLOCK,10,01/04/2018 18:42:02,7.57,8428852,French Southern Territories
9,21703,BAG 125g SWIRLY MARBLES,-7,01/05/2018 06:02:18,0.62,3080538,Yemen


In [ ]:
!sleep 60

In [ ]:
!unzip access.log.zip
!mkdir -p /var/log/httpd/
!mv access.log /var/log/httpd/

Archive:  access.log.zip
  inflating: access.log              
  inflating: client_hostname.csv     


## Check Logs

In [ ]:
!tail /var/log/aws-kinesis-agent/aws-kinesis-agent.log  

2021-05-04 19:15:02.410+0000 704287a1c18b (FileTailer[fh:tf-cadabra_batch_sales:/data/*.log].MetricsEmitter RUNNING) com.amazon.kinesis.streaming.agent.tailing.FileTailer [INFO] FileTailer[fh:tf-cadabra_batch_sales:/data/*.log]: Tailer Progress: Tailer has parsed 2500 records (213537 bytes), transformed 0 records, skipped 0 records, and has successfully sent 2500 records to destination.
2021-05-04 19:15:02.413+0000 704287a1c18b (FileTailer[kinesis:tf-order-stream:/data/*.log].MetricsEmitter RUNNING) com.amazon.kinesis.streaming.agent.tailing.FileTailer [INFO] FileTailer[kinesis:tf-order-stream:/data/*.log]: Tailer Progress: Tailer has parsed 2500 records (213537 bytes), transformed 2500 records, skipped 0 records, and has successfully sent 2500 records to destination.
2021-05-04 19:15:02.414+0000 704287a1c18b (FileTailer[fh:tf-weblogs:/var/log/httpd/*.log].MetricsEmitter RUNNING) com.amazon.kinesis.streaming.agent.tailing.FileTailer [INFO] FileTailer[fh:tf-weblogs:/var/log/httpd/*.log]

# Run Glue Crawler using aws CLI

In [ ]:
%cd {HOME}

/root


In [ ]:
!aws --version

aws-cli/2.2.1 Python/3.8.8 Linux/4.19.112+ exe/x86_64.ubuntu.18 prompt/off


In [ ]:
!aws glue start-crawler --name tf-cadabra_crawler

In [ ]:
!sleep 130

In [ ]:
!wget https://raw.githubusercontent.com/tadinve/rsa_on_aws/main/src/python_code/glue_schema_rename.py 

--2021-05-04 19:25:45--  https://raw.githubusercontent.com/tadinve/rsa_on_aws/main/src/python_code/glue_schema_rename.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2542 (2.5K) [text/plain]
Saving to: ‘glue_schema_rename.py’

glue_schema_rename. 100%[===================>]   2.48K  --.-KB/s    in 0s      

2021-05-04 19:25:45 (38.7 MB/s) - ‘glue_schema_rename.py’ saved [2542/2542]



In [ ]:
from glue_schema_rename import rename_glue_schema
rename_glue_schema('tf-cadabra_glue','salestransactions')

{'Name': 'invoiceno', 'Type': 'bigint'}
{'Name': 'stockcode', 'Type': 'string'}
{'Name': 'description', 'Type': 'string'}
{'Name': 'quantity', 'Type': 'bigint'}
{'Name': 'invoicedate', 'Type': 'string'}
{'Name': 'unitprice', 'Type': 'double'}
{'Name': 'customerid', 'Type': 'bigint'}
{'Name': 'country', 'Type': 'string'}
{'Name': 'year', 'Type': 'string'}
{'Name': 'month', 'Type': 'string'}
{'Name': 'day', 'Type': 'string'}
{'Name': 'hour', 'Type': 'string'}
Rename Complete


# Generate Huge Data

In [ ]:
#!aws s3 rm --recursive s3://tf-{NAME_INITIALS}-cadabra
#!aws s3 rm --recursive s3://tf-{NAME_INITIALS}-failed-http-logs
#!aws s3 rm --recursive s3://tf-{NAME_INITIALS}-redshift-stage-sales

delete: s3://tf-vg-cadabra/SalesTransactions/2021/05/04/19/tf-cadabra_batch_sales-1-2021-05-04-19-13-39-4b4b2179-baa7-4432-930a-dfc5bc232938
delete: s3://tf-vg-redshift-stage-sales/2021/05/04/19/tf-redshift-deliverystream-1-2021-05-04-19-13-39-b02a4bc6-d9c3-438c-8e62-7ba915bfe92d


In [ ]:
from transaction_generator import generateTransactions
one_m_file = TRANSACTIONS_DIR+"test_1M.log"
generateTransactions(n=int(10**4.1),OutFile=one_m_file)

In [ ]:
!ls -alh /data/

In [ ]:
def log_1g(n=3):
    suffix = ["1M","10M","100M","1G","10G","100G","1TB"]
    f = open(one_m_file)
    lines = f.read()
    f_out = open("/data/test_" + suffix[n] + ".csv","w")
    for i in range(10**n):
        f_out.write(lines)
    f_out.close()


In [ ]:
for i in range(1,5):
    log_1g(i)

In [ ]:
!ls -alh /data/

In [ ]:
!mv /data/test_10M.csv /data/test_10M.log

In [ ]:
import pprint

In [ ]:
import boto3

client = boto3.client('dynamodb','us-west-2')
table = dynamoDBClient.describe_table(TableName='tf-CadabraOrders')
pprint.pprint(table)

In [ ]:
import boto3

dynamoDBResource = boto3.resource('dynamodb')
table = dynamoDBResource.Table('tf-CadabraOrders')
print(table.item_count)

In [ ]:
client = boto3.client('dynamodb','us-west-2')
response = client.describe_table(TableName='tf-CadabraOrders')
print(response['Table']['ItemCount'])